In [ ]:
%matplotlib inline
import pymc3 as pm
import theano.tensor as T
import theano
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,StandardScaler
import statsmodels.api as sm

import pickle

pd.options.display.max_columns=150
pd.options.display.max_rows=150

In [ ]:
df = pd.read_csv('banking_sector_weekly.csv')
df.dropna(subset=['vol', 'eps','weekly_return', 'prior_wk_ret1', 'prior_wk_ret2',
       'prior_wk_ret3', 'prior_wk_ret4', 'prior_wk_ret5', 'prior_wk_ret6',
       'prior_wk_ret7', 'imports', 'exports', 'payroll', 'unemployment',
       'unemployment_delta'],inplace=True)
df['beg_date'] = pd.to_datetime(df['beg_date'])

df['ML3_return'] = (df['prior_wk_ret1']+df['prior_wk_ret2']+df['prior_wk_ret3'])/3.0
df['ML5_return'] = (df['prior_wk_ret1']+df['prior_wk_ret2']+df['prior_wk_ret3']+df['prior_wk_ret4']+df['prior_wk_ret5'])/5.0

scaler = StandardScaler()
df[['vol', 'eps', 'prior_wk_ret1', 'prior_wk_ret2',
       'prior_wk_ret3', 'prior_wk_ret4', 'prior_wk_ret5', 'prior_wk_ret6',
       'prior_wk_ret7', 'imports', 'exports', 'payroll', 'unemployment',
       'unemployment_delta']] = scaler.fit_transform(df[['vol', 'eps', 'prior_wk_ret1', 'prior_wk_ret2',
       'prior_wk_ret3', 'prior_wk_ret4', 'prior_wk_ret5', 'prior_wk_ret6',
       'prior_wk_ret7', 'imports', 'exports', 'payroll', 'unemployment',
       'unemployment_delta']])

df

#### Let's first try fully pooled to see what's up

In [ ]:
train_df = df[(df['beg_date']>pd.to_datetime('2014-01-01'))&(df['beg_date']<=pd.to_datetime('2017-01-01'))]
test_df = df[(df['beg_date']>=pd.to_datetime('2017-01-01'))]

x = train_df[['eps', 'prior_wk_ret1', 'prior_wk_ret2',
       'prior_wk_ret3', 'prior_wk_ret4', 'prior_wk_ret5', 'prior_wk_ret6',
       'prior_wk_ret7', 'imports', 'exports', 'payroll', 'unemployment',
       'unemployment_delta','ML3_return','ML5_return']]

x = sm.add_constant(x)

y = train_df[['weekly_return']]

lm = sm.OLS(y,x)
results = lm.fit()

results.summary()

In [ ]:
train_df['pred'] = results.predict(x)

train_df.to_csv('ols_trainset_results.csv',index=False)

In [ ]:
x = test_df[['eps', 'prior_wk_ret1', 'prior_wk_ret2',
       'prior_wk_ret3', 'prior_wk_ret4', 'prior_wk_ret5', 'prior_wk_ret6',
       'prior_wk_ret7', 'imports', 'exports', 'payroll', 'unemployment',
       'unemployment_delta','ML3_return','ML5_return']]

x = sm.add_constant(x)

test_df['pred'] = results.predict(x)

test_df.to_csv('ols_testset_results.csv',index=False)

#### Generalisation results are NAT good. Let us try partial pooling

In [ ]:
le = LabelEncoder()

df['conm_code'] = le.fit_transform(df.tic)
df['weekly_return'] = df['weekly_return'].astype(theano.config.floatX)


train_df = df[(df['beg_date']>pd.to_datetime('2014-01-01'))&(df['beg_date']<=pd.to_datetime('2017-01-01'))]
test_df = df[(df['beg_date']>=pd.to_datetime('2017-01-01'))]

print(train_df.shape)
print(test_df.shape)

relevant_vars=['eps', 'prior_wk_ret1', 'prior_wk_ret2',
       'prior_wk_ret3', 'prior_wk_ret4', 'prior_wk_ret5', 'prior_wk_ret6',
       'prior_wk_ret7', 'imports', 'exports', 'payroll', 'unemployment',
       'unemployment_delta','ML3_return','ML5_return']

conm_idx = train_df.conm_code.values
n_companies = len(df.conm.unique())

In [ ]:
conm_idx = theano.shared(np.array(train_df.conm_code.values,dtype='int32'))

x1_shared = theano.shared(np.array(train_df['prior_wk_ret1'],dtype='float32'))
x2_shared = theano.shared(np.array(train_df['prior_wk_ret2'],dtype='float32'))
x3_shared = theano.shared(np.array(train_df['prior_wk_ret3'],dtype='float32'))
x4_shared = theano.shared(np.array(train_df['prior_wk_ret4'],dtype='float32'))
x5_shared = theano.shared(np.array(train_df['prior_wk_ret5'],dtype='float32'))
x6_shared = theano.shared(np.array(train_df['prior_wk_ret6'],dtype='float32'))
x7_shared = theano.shared(np.array(train_df['prior_wk_ret7'],dtype='float32'))
x8_shared = theano.shared(np.array(train_df['eps'],dtype='float32'))
x9_shared = theano.shared(np.array(train_df['imports'],dtype='float32'))
x10_shared = theano.shared(np.array(train_df['exports'],dtype='float32'))
x11_shared = theano.shared(np.array(train_df['unemployment'],dtype='float32'))
x12_shared = theano.shared(np.array(train_df['unemployment_delta'],dtype='float32'))
x13_shared = theano.shared(np.array(train_df['ML3_return'],dtype='float32'))
x14_shared = theano.shared(np.array(train_df['ML5_return'],dtype='float32'))
#ann_output = theano.shared(train_df['weekly_return'])

In [ ]:
with pm.Model() as hierarchical_model:
    # Hyperpriors for group nodes
    mu_a = pm.Normal('mu_a', mu=0., sd=100000**2)
    sigma_a = pm.HalfCauchy('sigma_a', 50)
    mu_b = pm.Normal('mu_b', mu=0., sd=100000**2)
    sigma_b = pm.HalfCauchy('sigma_b', 50)
    mu_b1 = pm.Normal('mu_b1', mu=0., sd=100000**2)
    sigma_b1 = pm.HalfCauchy('sigma_b1', 50)
    mu_b2 = pm.Normal('mu_b2', mu=0., sd=100000**2)
    sigma_b2 = pm.HalfCauchy('sigma_b2', 50)
    mu_b3 = pm.Normal('mu_b3', mu=0., sd=100000**2)
    sigma_b3 = pm.HalfCauchy('sigma_b3', 50)
    mu_b4 = pm.Normal('mu_b4', mu=0., sd=100000**2)
    sigma_b4 = pm.HalfCauchy('sigma_b4', 50)
    mu_b5 = pm.Normal('mu_b5', mu=0., sd=100000**2)
    sigma_b5 = pm.HalfCauchy('sigma_b5', 50)
    mu_b6 = pm.Normal('mu_b6', mu=0., sd=100000**2)
    sigma_b6 = pm.HalfCauchy('sigma_b6', 50)
    mu_b7 = pm.Normal('mu_b7', mu=0., sd=100000**2)
    sigma_b7 = pm.HalfCauchy('sigma_b7', 50)
    mu_b8 = pm.Normal('mu_b8', mu=0., sd=100000**2)
    sigma_b8 = pm.HalfCauchy('sigma_b8', 50)
    mu_b9 = pm.Normal('mu_b9', mu=0., sd=100000**2)
    sigma_b9 = pm.HalfCauchy('sigma_b9', 50)
    mu_b10 = pm.Normal('mu_b10', mu=0., sd=100000**2)
    sigma_b10 = pm.HalfCauchy('sigma_b10', 50)
    mu_b11 = pm.Normal('mu_b11', mu=0., sd=100000**2)
    sigma_b11 = pm.HalfCauchy('sigma_b11', 50)
    mu_b12 = pm.Normal('mu_b12', mu=0., sd=100000**2)
    sigma_b12 = pm.HalfCauchy('sigma_b12', 50)
    mu_b13 = pm.Normal('mu_b13', mu=0., sd=100000**2)
    sigma_b13 = pm.HalfCauchy('sigma_b13', 50)
    #mu_b14 = pm.Normal('mu_b14', mu=0., sd=100**2)
    #sigma_b14 = pm.HalfCauchy('sigma_b14', 5)
    

    # Intercept for each conm, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_companies).
    a = pm.Normal('a', mu=mu_a, sd=sigma_a, shape=n_companies)
    # beta 1 through 6
    b = pm.Normal('b', mu=mu_b, sd=sigma_b, shape=n_companies)
    b1 = pm.Normal('b1', mu=mu_b1, sd=sigma_b1, shape=n_companies)
    b2 = pm.Normal('b2', mu=mu_b2, sd=sigma_b2, shape=n_companies)
    b3 = pm.Normal('b3', mu=mu_b3, sd=sigma_b3, shape=n_companies)
    b4 = pm.Normal('b4', mu=mu_b4, sd=sigma_b4, shape=n_companies)
    b5 = pm.Normal('b5', mu=mu_b5, sd=sigma_b5, shape=n_companies)
    b6 = pm.Normal('b6', mu=mu_b6, sd=sigma_b6, shape=n_companies)
    b7 = pm.Normal('b7', mu=mu_b7, sd=sigma_b7, shape=n_companies)
    b8 = pm.Normal('b8', mu=mu_b8, sd=sigma_b8, shape=n_companies)
    b9 = pm.Normal('b9', mu=mu_b9, sd=sigma_b9, shape=n_companies)
    b10 = pm.Normal('b10', mu=mu_b10, sd=sigma_b10, shape=n_companies)
    b11 = pm.Normal('b11', mu=mu_b11, sd=sigma_b11, shape=n_companies)
    b12 = pm.Normal('b12', mu=mu_b12, sd=sigma_b12, shape=n_companies)
    b13 = pm.Normal('b13', mu=mu_b13, sd=sigma_b13, shape=n_companies)
    #b14 = pm.Normal('b14', mu=mu_b14, sd=sigma_b14, shape=n_companies)

    # Model error
    #['eps', 'prior_wk_ret1', 'prior_wk_ret2',
    #   'prior_wk_ret3', 'prior_wk_ret4', 'prior_wk_ret5', 'prior_wk_ret6',
    #   'prior_wk_ret7', 'imports', 'exports', 'payroll', 'unemployment',
    #   'unemployment_delta','ML3_return','ML5_return']
    
    sigma = pm.Gamma("sigma", alpha=10, beta=1)
    
    #dir_est=a[conm_idx] + b[conm_idx] * x1_shared + b1[conm_idx] * x2_shared + \
    #         b2[conm_idx] * x3_shared + b3[conm_idx] * x4_shared + \
    #         b4[conm_idx] * x5_shared + b5[conm_idx] * x6_shared + \
    #        b6[conm_idx] * x7_shared + b7[conm_idx] * x8_shared + \
    #        b8[conm_idx] * x9_shared + b9[conm_idx] * x10_shared + \
    #        b10[conm_idx] * x11_shared + b11[conm_idx] * x12_shared + \
    #        b12[conm_idx] * x13_shared + b13[conm_idx] * x14_shared
    
    dir_est=a[conm_idx] + b[conm_idx] * x1_shared + b7[conm_idx] * x8_shared + \
            b8[conm_idx] * x9_shared + b9[conm_idx] * x10_shared + \
            b10[conm_idx] * x11_shared + b11[conm_idx] * x12_shared + \
            b12[conm_idx] * x13_shared + b13[conm_idx] * x14_shared


    # Data likelihood
    dir_like = pm.Normal('dir_like', mu=dir_est, sd=sigma, observed=train_df.weekly_return)
    #dir_like = pm.Logistic('dir_like', mu=dir_est, s=sigma, observed=data_train.weekly_direction)
    #dir_like = pm.Bernoulli('dir_like',p=dir_est, observed=data_train.weekly_direction)
    
    start = pm.find_MAP()

In [ ]:
start['b13']

In [ ]:
conm_idx = test_df.conm_code.values

preds = start['a'][conm_idx] + start['b'][conm_idx] * test_df['prior_wk_ret1'] + start['b7'][conm_idx] * test_df['eps'] + \
            start['b8'][conm_idx] * test_df['imports'] + start['b9'][conm_idx] * test_df['exports'] + \
            start['b10'][conm_idx] * test_df['unemployment'] + start['b11'][conm_idx] * test_df['unemployment_delta'] + \
            start['b12'][conm_idx] * test_df['ML3_return'] + start['b13'][conm_idx] * test_df['ML5_return']

test_df['preds'] = preds

test_df.to_csv('bhm_testset_results_MAP.csv',index=False)